In [143]:
import sklearn as sk
import numpy as np
import pandas as pd
import rasterio, os, napari
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.cluster import KMeans

In [153]:
with open(r"C:\Users\marke\Documents\DSC180A-Q1\testing_data\checkpoints\data.npy", "rb") as file:
    data = np.load(file)

In [155]:
with open(r"C:\Users\marke\Documents\DSC180A-Q1\testing_data\checkpoints\labels.npy", "rb") as file:
    labels = np.load(file)

In [156]:
labels

array(['Ice', 'Ice', 'Ice', ..., 'Not Ice', 'Not Ice', 'Not Ice'],
      dtype='<U32')

In [154]:
data.shape

(7658, 9)

In [94]:
labels.shape

(58737,)

In [157]:
 X_train, X_test, y_train, y_test = train_test_split(data,labels, test_size=0.2)

In [158]:
X_train

array([[1.62146568e-01, 1.66878968e-01, 1.84082046e-01, ...,
        2.87476715e+02, 1.70462698e-01, 1.83871254e-01],
       [3.76224101e-01, 4.23249632e-01, 4.00827229e-01, ...,
        2.71747742e+02, 3.45703885e-02, 7.36823678e-01],
       [3.76224101e-01, 4.23249632e-01, 4.00827229e-01, ...,
        2.62050262e+02, 3.51652443e-01, 6.45157754e-01],
       ...,
       [1.16272815e-01, 1.06657669e-01, 1.02598883e-01, ...,
        2.91664246e+02, 5.18265553e-02, 9.81192812e-02],
       [2.15665951e-01, 2.13335395e-01, 2.24823624e-01, ...,
        2.79252716e+02, 1.87718868e-01, 1.95699126e-01],
       [9.48650613e-02, 7.74073228e-02, 7.16352835e-02, ...,
        2.79581024e+02, 4.31984700e-02, 7.44635612e-02]])

In [159]:
a = pd.Series(y_train).value_counts()
b = pd.Series(y_test).value_counts()

print(a)
print("\n")
print(b)

Not Ice    3665
Ice        2461
dtype: int64


Not Ice    891
Ice        641
dtype: int64


### Random Forest

In [160]:
fclf = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)

In [161]:
fclf.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, max_features=1, n_estimators=10)

In [162]:
fclf.score(X_test, y_test)

0.9347258485639687

In [163]:
fclf.predict(X_test)


array(['Not Ice', 'Not Ice', 'Not Ice', ..., 'Not Ice', 'Ice', 'Not Ice'],
      dtype='<U32')

In [164]:
fclf.predict(X_test)

array(['Not Ice', 'Not Ice', 'Not Ice', ..., 'Not Ice', 'Ice', 'Not Ice'],
      dtype='<U32')

In [165]:
sklearn.metrics.f1_score(y_test,fclf.predict(X_test), pos_label='Not Ice')

0.9432463110102157

### SVM

In [ ]:
sclf = SVC(gamma='auto')

In [259]:
sclf.fit(X_train,y_train)

SVC(gamma='auto')

In [262]:
sclf.score(X_test, y_test)

0.8838120104438643

In [261]:
sklearn.metrics.f1_score(y_test,sclf.predict(X_test), pos_label='Not Ice')

0.8960280373831776

### Dimentionality Reduction

In [87]:
cclf = KMeans()

In [88]:
cclf.fit(X_train)

KMeans()

# Display prediction

In [166]:
test_img = "Engilchek_glacier_2002-08-18.tif"

In [167]:
full_directory = os.path.join(r"C:\Users\marke\Documents\DSC180A-Q1\raw_data\full_img",test_img)
ndsi_directory = os.path.join(r"C:\Users\marke\Documents\DSC180A-Q1\raw_data\ndsi_imgs",test_img)

In [132]:
full_directory

'C:\\Users\\marke\\Documents\\DSC180A-Q1\\raw_data\\full_img\\Engilchek_glacier_2002-08-18.tif'

In [168]:
def loadImage(img_path, full_img_path, ndsi_directory):
    with rasterio.open(full_directory) as src:
        full_img = src.read()
    
    with rasterio.open(ndsi_directory) as src:
        ndsi_img = src.read()

    return full_img, ndsi_img

In [264]:
def loadAndTest(test_img,full_directory,ndsi_directory, trained_model):
    full,ndsi = runNapari(test_img,full_directory,ndsi_directory)
    
    e_dims = full.shape
    testbals = np.rollaxis(full.reshape(9,e_dims[1]*e_dims[2]),0,2)
    prediction = trained_model.predict(testbals)

    prediction = prediction.reshape(e_dims[1],e_dims[2])
    # np.count_nonzero(testbals.reshape(9,145,363) != full)
    display_dims = prediction.shape
    img_coordinates = np.array(list(zip(np.arange(0,display_dims[0]),np.arange(0,display_dims[1]))))
    
    ice_indexes = np.where(prediction == "Ice")
    non_ice_indexes = np.where(prediction == "Not Ice")
    ice_coordinates = list(zip(ice_indexes[0],ice_indexes[1]))
    non_ice_coordinates = list(zip(non_ice_indexes[0], non_ice_indexes[1]))
    
    viewer = napari.view_image(ndsi)
    viewer.add_points(ice_coordinates,face_color="red",edge_color ="red",size=1, name="Ice")
    viewer.add_points(non_ice_coordinates,face_color="blue",edge_color ="blue",size=1, name="Not Ice")


In [265]:
loadAndTest(test_img,full_directory,ndsi_directory, sclf)

In [169]:
full,ndsi = runNapari(test_img,full_directory,ndsi_directory)

In [247]:
e_dims = full.shape
testbals = np.rollaxis(full.reshape(9,e_dims[1]*e_dims[2]),0,2)
prediction = fclf.predict(testbals)

prediction = prediction.reshape(e_dims[1],e_dims[2])
# np.count_nonzero(testbals.reshape(9,145,363) != full)
display_dims = prediction.shape
display_dims

(145, 363)

In [239]:
img_coordinates = np.array(list(zip(np.arange(0,display_dims[1]),np.arange(0,display_dims[2]))))

In [250]:
ice_indexes = np.where(prediction == "Ice")
non_ice_indexes = np.where(prediction == "Not Ice")

In [251]:
ice_coordinates = list(zip(ice_indexes[0],ice_indexes[1]))
non_ice_coordinates = list(zip(non_ice_indexes[0], non_ice_indexes[1]))

In [253]:
viewer = napari.view_image(ndsi)

In [255]:
viewer.add_points(ice_coordinates,face_color="red",edge_color ="red",size=1, name="Ice")
viewer.add_points(non_ice_coordinates,face_color="blue",edge_color ="blue",size=1, name="Not Ice")


<Points layer 'Not Ice' at 0x21aaee09520>